In [ ]:
import requests
from bs4 import BeautifulSoup

def get_all_tag():
    latest = 'https://finance.yahoo.com/news/'
    response = requests.get(latest)
    data = BeautifulSoup(response,"html.parser")
    news = data.find_all(attrs={"class": "js-stream-content Pos(r)"})
    return news

In [ ]:
def header_and_link(tag):
    dictionary = {}
    for i in tag:
        header = i.find("a")
        link = header['href']
        if 'https' not in link:
            link = "https://finance.yahoo.com"+link
        dictionary[header.text] = link
    return dictionary

In [1]:
import requests
from bs4 import BeautifulSoup

latest = 'https://www.infoquest.co.th/2022/243941'
response = requests.get(latest).text
data = BeautifulSoup(response,"html.parser")
body = data.find('body').find()
print(body.text)







 Skip to content




 

สำนักข่าวอินโฟเควสท์ 

 

 

 เลือกตั้ง’66
ข่าวเศรษฐกิจ
ข่าวหุ้น
ข่าวต่างประเทศ
ข่าวธุรกิจ
ข่าวทั่วไป
About

dataxet:infoquest
About Us
Contact Us


 






เลือกตั้ง’66
ข่าวเศรษฐกิจ
ข่าวหุ้น
ข่าวต่างประเทศ
ข่าวธุรกิจ
ข่าวทั่วไป
About

dataxet:infoquest
About Us
Contact Us


 






Search for:



 







PTT แจงยังไม่ได้รับหนังสือลาออกของประธานบอร์ด 


 ข่าวหุ้น20 ต.ค. 65 8:49น.2022-10-20 
Facebook iconFacebookTwitter iconTwitterLINE iconLine

บมจ.ปตท (PTT) ชี้แจงกรณีมีข่าวในสื่อบางแห่งเกี่ยวกับการลาออกของนายทศพร ศิริสัมพันธ์ ประธานกรรมการและกรรมการอิสระ ปตท. นั้นว่า ปตท. ยังไม่ได้รับหนังสือลาออกจากตำแหน่งประธานกรรมการและกรรมการอิสระ ของนายทศพร แต่อย่างใด ทั้งนี้หาก ปตท. ได้รับหนังสือดังกล่าว หรือมีข้อมูลความคืบหน้าที่เป็นสาระสำคัญในประเด็นนี้เพิ่มเติม ปตท. จะได้แจ้งต่อตลาดหลักทรัพย์แห่งประเทศไทยต่อไป
ขณะที่ ปตท.แจ้งว่า นางเปรมฤทัย วินัยแพทย์ กรรมการและกรรมการบริหารความเสี่ยงองค์กร ได้แจ้งลาออกจากตำแหน่งกรรมการ ปตท. เนื่องจากมีภารกิจอื่น โดยการลาออกให้มี

In [ ]:
def header_and_content(dictionary):
    new_dict = {}
    for i in list(dictionary.keys()):
        try:
            response = requests.get(dictionary[i])
            data = BeautifulSoup(response.content,"html.parser")
            content = data.find(attrs={"class": "caas-body"}).find_all('p')
            content_text = ""
            for j in content:
                if '@' in j.text:
                    break
                content_text += j.text+'\n'
            new_dict[i]=content_text
        except AttributeError:      
            new_dict[i]=dictionary[i]
    return new_dict

In [ ]:
tag = get_all_tag()
handl = header_and_link(tag)
handc = header_and_content(handl)

In [ ]:
from data.function import News
new = News()
new.fetch_news()

In [ ]:
####################################

In [ ]:
import requests
from bs4 import BeautifulSoup
from googletrans import Translator

def get_all_tags(page):
    latest = f'https://www.kaohoon.com/latest-news/page/{page}'
    response = requests.get(latest)
    data = BeautifulSoup(response.text,"html.parser")
    news = data.find_all(attrs={"class": "tie-row"})
    return news[4:14]

In [ ]:
"tie-col-md-1 tie-col-sm-2 tie-col-xs-2"#time
"tie-col-md-11 tie-col-sm-10 tie-col-xs-10"#header
def header_link_time(tag):
    # .replace('\n','').replace('“',"").replace('”','')
    result = []
    for data in tag:
        time_tag = data.find(attrs={"class": "tie-col-md-1 tie-col-sm-2 tie-col-xs-2"})
        header_tag = data.find(attrs={"class": "tie-col-md-11 tie-col-sm-10 tie-col-xs-10"})
        header = header_tag.find('a')
        time = time_tag.find('h5').text
        date = time_tag.find('h6').text.split('/')
        result.append([header.text,f"{date[2]}-{date[1]}-{date[0]} {time}:00",header["href"]])
    return result

In [ ]:
def content(link):
    text = ''
    response = requests.get(link)
    data = BeautifulSoup(response.text,"html.parser")
    try:
        news = data.find(attrs={"itemprop": "articleBody"}).find_all('p')
    except AttributeError:
        return 'null'
    for i in news:
        text = text+i.text+'\n'
    if text == '':
        return 'null'
    return text
    

In [ ]:
def news_list(l):
    data = []
    for i in l:
        con = content(i[2])
        data.append(i+[con])
    return data

In [ ]:
import sqlite3
def insert_news(news):
    conn = sqlite3.connect('stock.db',timeout=10)#connect to database
    cursor = conn.cursor()
    cursor.execute(f"INSERT INTO news VALUES (null,?,?,?,?)", (news[0],news[1],news[2],news[3]))
    conn.commit()
    conn.close()

def get_title():
    conn = sqlite3.connect('stock.db',timeout=10)
    cursor = conn.cursor()
    cursor.execute(f"SELECT title FROM news")
    data = cursor.fetchall()
    conn.close()
    return [i[0] for i in data]

In [ ]:
all_data = []
i = 0
fetch_latest = False
while(i<10000):
    i += 1
    has_title = get_title()
    data = news_list(header_link_time(get_all_tags(i)))
    for j in data:
        if j[0] in has_title:
            # fetch_latest = True
            # break
            continue
        insert_news(j)
        print('---------------------100%-----------------------')

In [ ]:
from pythainlp.tokenize import word_tokenize
word_tokenize('บริษัทดำเนินธุรกิจ และเติบโตคู่กับคนไทยมากว่า 34 ปี ซึ่งบริษัทฯ มีความตระหนักดีในการให้ความช่วยเหลือสังคม ส่งเสริมคุณภาพชีวิตของชุมชนในพื้นที่ที่ PTG ดำเนินกิจการอยู่ จึงจัดกิจกรรม CSR ภายใต้โครงการ “PT ค่ายอาสา ทำจริงไม่ทิ้งกัน” มาอย่างต่อเนื่อง ล่าสุด PTG ได้เริ่มโครงการฯ แรกของปี และจัดขึ้นเป็นครั้งที่ 8 ในวันที่ 18 กุมภาพันธ์ 2566 ณ โรงเรียนวัดใหม่ราษฎร์ศรัทธาราม ตำบลบึงพระ อำเภอเมือง จังหวัดพิษณุโลก เพื่อให้เยาวชน และชาวบ้านผู้อาศัยในตำบลบึงพระ จำนวน 10 หมู่บ้าน ได้รับประโยชน์จากกิจกรรมในครั้งนี้สำหรับกิจกรรมครั้งนี้ PTG ได้ร่วมมือกับ คลังน้ำมันพีทีพิษณุโลก เพราะทาง PTG มีความตั้งใจเป็นอย่างมากที่จะสร้างสรรค์กิจกรรมสาธารณะประโยชน์ต่อชุมชนกับโครงการ “PT ค่ายอาสา ทำจริงไม่ทิ้งกัน” โดยมีกิจกรรมหลักคือโครงการ “พีที ชุมชนตาสว่าง” ซึ่งเป็นกิจกรรมส่งเสริมสุขภาพอนามัย พัฒนาด้านความเป็นอยู่ของคนในชุมชน โดยการบริการวัดสายตาประกอบแว่น และให้ความรู้เรื่องดวงตากับผู้สูงอายุ ซึ่งปัจจุบัน PTG มอบแว่นตาตัดใหม่ทุกอันให้ผู้สูงอายุไปแล้วกว่า 2,800 คน และที่ตำบลบึงพระได้เพิ่มอีก จำนวน 500 คน เพื่อให้ทุกท่านมีชีวิตความเป็นอยู่ที่ดีขึ้น และนำความรู้ที่ได้ไปใช้ประโยชน์ในชีวิตประจำวันอย่างยั่งยืนทั้งนี้ โครงการฯ ยังได้ให้ความสำคัญกับการพัฒนาคุณภาพการศึกษา ให้ความช่วยเหลือและสนับสนุนโรงเรียนที่อยู่ในพื้นที่ตำบลบึงพระ อำเภอเมือง จังหวัดพิษณุโลก จำนวน 3 สถานศึกษา และมอบทุนการศึกษาให้กับนักเรียนที่เรียนดีแต่ขาดแคลนทุนทรัพย์ จำนวน 40 ทุน แบ่งเป็นทุนละ 2,000 บาท และมอบทุนการศึกษาบุตรและธิดาของพนักงาน ที่เรียนดีแต่ขาดแคลนทุนทรัพย์ อีกจำนวน 10 ทุน แบ่งเป็นทุนละ 2,000 บาทนอกจากนี้ ยังมีการมอบอุปกรณ์การเรียนการสอน อุปกรณ์กีฬา กล่องปฐมพยาบาล พร้อมพันธุ์ผักสวนครัวเพื่อให้น้อง ๆ เพื่อใช้ในการประกอบอาหารกลางวัน ทั้ง 3 สถานศึกษา พร้อมทั้งยังร่วมกับคลังน้ำมันพีทีพิษณุโลก ปรับปรุงทาสีรั้วโรงเรียน และมอบหนังสือส่งเสริมพัฒนาการ สื่อการเรียน พร้อมปรับปรุงห้องสมุดเพื่อน้อง ให้กับโรงเรียนวัดใหม่ราษฎร์ศรัทธาราม ทั้งนี้ PTG ยังพร้อมส่งเสริมทางด้านสิ่งแวดล้อม และสนับสนุนโภชนาการน้อง ๆ โดยการมอบแปลงผักสวนครัวเพื่อน้อง โดยร่วมกันปลูกผักสวนครัว ไม้กินได้ และปลูกป่าสร้างปอด รวมจำนวนกว่า 200 ต้น ให้กับโรงเรียนวัดใหม่ราษฎร์ศรัทธาราม เพื่อให้น้องๆ ได้นำมาประกอบอาหารกลางวันในอนาคต รวมถึงมีกิจกรรมต้นไม้แลกยิ้ม ที่มอบต้นไม้กินได้ จำนวนกว่า 300 ต้น เพื่อให้ชาวบ้านได้นำต้นไม้กลับไปปลูกที่บ้านต่อไปนายรังสรรค์ กล่าวอีกว่า การจัดกิจกรรมครั้งนี้พิเศษขึ้นไปอีก เนื่องจาก PTG นำอาสาสมัคร Gen Z ในจังหวัดพิษณุโลก มาร่วมกิจกรรม PT พี่อาสา สอนน้องเรียน พาน้องเล่น PT Gen Z English Camp เสริมสร้างความรู้สอดแทรกกิจกรรมนันทนา การภาษาอังกฤษให้กับน้องๆ และสร้างสีสันกับเพลงเพราะๆ กับกิจกรรม PT Gen Z Music Concert ให้ทุกคนได้เพลิดเพลินกันตลอดทั้งงาน ตรงนี้จะช่วยให้คนรุ่นใหม่ซึมซับกับการช่วยเหลือสังคมและนำไปต่อยอดให้กิจกรรมมีความยั่งยืนต่อไปในอนาคตโดยนอกเหนือจากกิจกรรมดังกล่าว PTG ยังความคำนึงถึงความปลอดภัยในการขับขี่ของคนในชุมชน ดังนั้นจึงมีการจัดอบรมการขับขี่ปลอดภัยให้กับชาวบ้านในตำบลบึงพระ โดยวิทยากรด้านการจราจรพร้อมแจกหมวกกันน็อคให้กับผู้เข้าร่วมอบรม ได้ขับขี่อย่างถูกกฎจราจรและปลอดภัยอีกด้วย รวมทั้งยังลงพื้นที่เยี่ยมเยียนผู้ป่วยติดเตียง พร้อมมอบของอุปโภคบริโภค เพื่อใช้ในการดำเนินชีวิตประจำวัน จำนวน 10 ราย รวมทั้งจัดกิจกรรมปล่อยพันธุ์ปลาน้ำจืดกว่า 3,000 ตัว เพื่อให้ชาวบ้านในชุมชนสามารถจับไปรับประทานและสร้างรายได้เพื่อดำรงชีวิตต่อไป“PTG มุ่งเน้นใส่ใจความเป็นอยู่ของผู้คนในชุมชนแบบทุกมิติ ดังนั้นจึงเห็นมาโดยตลอดว่า PTG ได้จัดกิจกรรมสาธารณะประโยชน์เพื่อให้ความช่วยเหลือและสนับสนุนตามความจำเป็นในหลากหลายด้าน เพื่อบรรเทาทุกข์ และช่วยส่ง เสริมคุณภาพชีวิตให้กับคนในสังคม โดยยึดหลัก PT สร้างสุข สุขกาย สุขใจ สังคมอยู่ร่วมกันอย่าง “อยู่ดีมีสุข” ตลอดไปและยั่งยืน” นายรังสรรค์ กล่าวทิ้งท้าย',engine='newmm')

In [ ]:
import sqlite3
def get_all_title_content():
    conn = sqlite3.connect('stock.db',timeout=10)
    cursor = conn.cursor()
    cursor.execute(f"SELECT news_id,title,content FROM news")
    data = cursor.fetchall()
    conn.close()
    return data

In [ ]:
get_all_title_content()

In [2]:
from googletrans import Translator

def translate_text(text):
    translator = Translator()
    clean_text = text.replace('“',"").replace('”','')
    translated = translator.translate(clean_text).text
    return translated

def translate_paragraph(paragraph):
    array = paragraph.split('\n')
    translated = ''
    for i in array:
        translated += translate_text(i)
    return translated

def detect(text):
    translator = Translator()
    return translator.detect(text).lang

In [1]:
from data.function import News
news = News('set')
news.combine_translate(1)

"SUPER joins forces with PTT Group to buy and sell REC to expand customer base - reduce greenhouse gas emissions Mr. Chaphamon Chantarapongphan Managing Director of Super Carbon X Co., Ltd. or SCX, a subsidiary of Super Energy Corporation Public Company Limited or SUPER, revealed that he had signed a contract with Mekha V Co., Ltd. or Mekha V, a subsidiary of the company. PTT Public Company Limited or PTT in the sale of RECs renewable energy power generation certificates. In the past, SCX and Mekha V have entered into a large number of I-REC purchase and sale agreements. And aiming to increase the trading volume with Mekha V in the future many times more according to the needs of customers. SCX is a procurement and consolidator of environmental products. Environmental Products such as I-REC, TVER and other standard carbon to meet the needs of customers in the industrial sector Currently, SCX has the capacity to issue over 700,000 RECs from solar power plants under SUPER's conglomerate 

In [24]:
from googletrans import Translator
translator = Translator()
text = "บริษัท ปตท. จำกัด (มหาชน) หรือ PTT เปิดเผย ถึงผลการดำเนินงานงวดไตรมาส 4/65 บริษัทมีกำไรสุทธิ จำนวน 17,872 ล้านบาท หรือมากกว่าร้อยละ 100.0 จากไตรมาส 3/65 อยู่ที่ 8,884 ล้านบาท"
print(translator.translate(text).text)

PTT Public Company Limited or PTT disclosed the operating results for the 4th quarter of 2022, the company had a net profit of 17,872 million baht or more than 100.0 percent from the 3rd quarter of 2022 at 8,884 million baht.


In [202]:
"""set news"""
import requests
from bs4 import BeautifulSoup
from googletrans import Translator

def set_get_all_tags(symbol,page):
    latest = f'https://www.kaohoon.com/tag/{symbol}/page/{page}'
    response = requests.get(latest)
    if response.status_code == 200:
        data = BeautifulSoup(response.text,"html.parser")
        news = data.find_all(attrs={"class": "post-item"})
        return news
    return 'Server is not response'

def set_title_link_time(tag):
    # .replace('\n','').replace('“',"").replace('”','')
    result = []
    for data in tag:
        time_tag = data.find(attrs={"class": "date meta-item tie-icon"})
        title_tag = data.find(attrs={"class": "post-title"})
        title = title_tag.find('a')
        date = time_tag.text.split('/')
        result.append({'title':title.text,'datetime':f"{date[2]}-{date[1]}-{date[0]} 00:00:00",'link':title["href"]})
    return result

def set_content(link):
    text = ''
    response = requests.get(link)
    if response.status_code == 200:
        data = BeautifulSoup(response.text,"html.parser")
        try:
            news = data.find(attrs={"itemprop": "articleBody"}).find_all('p')
        except AttributeError:
            return 'null'
        for i in news:
            text = text+i.text+'\n'
        if text == '':
            return 'null'
        return text
    return 'null'

def set_news_dict(li,symbol):
    dict_list = []
    for news in li:
        con = set_content(news['link'])
        news['stock'] = symbol.upper()
        news['content'] = con
        dict_list.append(news)
    return dict_list

# def fetch_set_news(symbol):
#     all_news = []
#     page = 1
#     tag = set_get_all_tags(symbol,page)
#     while(tag != 'Server is not response'):
#         dict_list = header_link_time(tag)
#         result = set_news_dict(dict_list,symbol)
#         all_news += result
#         page += 1
#         tag = set_get_all_tags(symbol,page)
#     return all_news

#################################

import sqlite3
def insert_set_news(news):
    conn = sqlite3.connect('stock.db',timeout=10)#connect to database
    cursor = conn.cursor()
    cursor.execute("INSERT INTO set_news VALUES (null,?,?,?,?)", (news['title'],news['datetime'],news['link'],news['content']))
    conn.commit()
    conn.close()

def get_stock_id(symbol):
    """return id of stock"""       
    conn = sqlite3.connect('stock.db',timeout=10)
    cursor = conn.cursor()
    cursor.execute(f"SELECT stock_id FROM stock WHERE symbol = '{symbol.upper()}'")
    data = cursor.fetchall()
    conn.close()
    if len(data) != 0:
        return data[0][0]
    return None

def get_set_news_id(title):
    conn = sqlite3.connect('stock.db',timeout=10)
    cursor = conn.cursor()
    cursor.execute("SELECT news_id FROM set_news WHERE title = ?",(title,))
    data = cursor.fetchall()
    conn.close()
    if len(data) != 0:
        return data[0][0]
    return None

def check_relation(stock_id,news_id):
    conn = sqlite3.connect('stock.db',timeout=10)
    cursor = conn.cursor()
    cursor.execute(f"SELECT news_id FROM many_set_news WHERE stock_id = {stock_id}")
    data = cursor.fetchall()
    conn.close()
    news = []
    if len(data) != 0:
        news = [i[0] for i in data]
    return news_id in news
    

def get_all_title():
    conn = sqlite3.connect('stock.db',timeout=10)
    cursor = conn.cursor()
    cursor.execute(f"SELECT title FROM set_news")
    data = cursor.fetchall()
    conn.close()
    if len(data) != 0:
        return [i[0] for i in data]
    return []

def insert_many_set_news(news_id,stock_id):
    conn = sqlite3.connect('stock.db',timeout=10)#connect to database
    cursor = conn.cursor()
    cursor.execute(f"INSERT INTO many_set_news VALUES ({news_id},{stock_id})")
    conn.commit()
    conn.close()

def insert_set_many_news(news):
    # news = fetch_set_news(symbol)
    # for i in news:
    all_title = get_all_title()
    stock_id = get_stock_id(news['stock'])
    if news['title'] in all_title:
        news_id = get_set_news_id(news['title'])
        if not check_relation(stock_id,news_id):
            insert_many_set_news(news_id,stock_id)
    else:
        insert_set_news(news)
        news_id = get_set_news_id(news['title'])
        insert_many_set_news(news_id,stock_id)
    print('-----------------100%------------------------')


def fetch_set_news(symbol):
    page = 1
    tag = set_get_all_tags(symbol,page)
    while(tag != 'Server is not response'):
        dict_list = set_title_link_time(tag)
        result = set_news_dict(dict_list,symbol)
        for i in result:
            insert_set_many_news(i)
        page += 1
        tag = set_get_all_tags(symbol,page)
    
    


        

In [204]:
fetch_set_news('aot')


-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%-------------

In [205]:
"""nasdaq news"""
import requests

def nasdaq_get_all_tag(symbol):
    latest = f'https://finance.yahoo.com/quote/{symbol.upper()}'
    response = requests.get(latest)
    if response.status_code == 200:
        data = BeautifulSoup(response.text,"html.parser")
        tags = data.find_all(attrs={"class": "js-stream-content Pos(r)"})
        return tags
    return 'Server is not response'

def nasdaq_title_link(tag):
    result = []
    for i in tag:
        title = i.find("a")
        link = title['href']
        if 'https' not in link:
            link = 'https://finance.yahoo.com'+link
        result.append({'title':title.text,'link':link})
    return result

def nasdaq_content_time(link):
    content_dict = {}
    content_text = ""
    response = requests.get(link)
    if response.status_code == 200:
        data = BeautifulSoup(response.text,"html.parser")
        try:
            time = data.find('time')
            content = data.find(attrs={"class": "caas-body"}).find_all('p')
            content_text = ""
            for j in content:
                if '@' in j.text:
                    break
                content_text += j.text+'\n'
            content_dict['content'] = content_text
            content_dict['datetime'] = time["datetime"].replace('T',' ').replace('.000Z','')
        except AttributeError:      
            content_dict['content'] = "null"
        return content_dict
    content_dict['content'] = "null"
    content_dict['datetime'] = "null"
    return content_dict

def nasdaq_news_dict(li,symbol):
    dict_list = []
    for news in li:
        con = nasdaq_content_time(news['link'])
        news['stock'] = symbol.upper()
        news['datetime'] = con['datetime']
        news['content'] = con['content']
        dict_list.append(news)
    return dict_list

# def fetch_nasdaq_news(symbol):
#     tag = nasdaq_get_all_tag(symbol)
#     if tag == 'Server is not response':
#         return None
#     dict_list = nasdaq_title_link(tag)
#     result = nasdaq_news_dict(dict_list,symbol)
#     return result


import sqlite3
def insert_nasdaq_news(news):
    conn = sqlite3.connect('stock.db',timeout=10)#connect to database
    cursor = conn.cursor()
    cursor.execute("INSERT INTO nasdaq_news VALUES (null,?,?,?,?)", (news['title'],news['datetime'],news['link'],news['content']))
    conn.commit()
    conn.close()

def get_stock_id(symbol):
    """return id of stock"""       
    conn = sqlite3.connect('stock.db',timeout=10)
    cursor = conn.cursor()
    cursor.execute(f"SELECT stock_id FROM stock_nasdaq WHERE symbol = '{symbol.upper()}'")
    data = cursor.fetchall()
    conn.close()
    if len(data) != 0:
        return data[0][0]
    return None

def get_nasdaq_news_id(title):
    conn = sqlite3.connect('stock.db',timeout=10)
    cursor = conn.cursor()
    cursor.execute("SELECT news_id FROM nasdaq_news WHERE title = ?",(title,))
    data = cursor.fetchall()
    conn.close()
    if len(data) != 0:
        return data[0][0]
    return None

def check_relation(stock_id,news_id):
    conn = sqlite3.connect('stock.db',timeout=10)
    cursor = conn.cursor()
    cursor.execute(f"SELECT news_id FROM many_nasdaq_news WHERE stock_id = {stock_id}")
    data = cursor.fetchall()
    conn.close()
    news = []
    if len(data) != 0:
        news = [i[0] for i in data]
    return news_id in news
    

def get_all_title():
    conn = sqlite3.connect('stock.db',timeout=10)
    cursor = conn.cursor()
    cursor.execute(f"SELECT title FROM nasdaq_news")
    data = cursor.fetchall()
    conn.close()
    if len(data) != 0:
        return [i[0] for i in data]
    return []

def insert_many_nasdaq_news(news_id,stock_id):
    conn = sqlite3.connect('stock.db',timeout=10)#connect to database
    cursor = conn.cursor()
    cursor.execute(f"INSERT INTO many_nasdaq_news VALUES ({news_id},{stock_id})")
    conn.commit()
    conn.close()

def insert_nasdaq_many_news(news):
    # news = fetch_set_news(symbol)
    # for i in news:
    all_title = get_all_title()
    stock_id = get_stock_id(news['stock'])
    if news['title'] in all_title:
        news_id = get_nasdaq_news_id(news['title'])
        if not check_relation(stock_id,news_id):
            insert_many_nasdaq_news(news_id,stock_id)
    else:
        insert_nasdaq_news(news)
        news_id = get_nasdaq_news_id(news['title'])
        insert_many_nasdaq_news(news_id,stock_id)
    print('-----------------100%------------------------')


def fetch_nasdaq_news(symbol):

    tag = nasdaq_get_all_tag(symbol)
    if tag == 'Server is not response':
        return None
    dict_list = nasdaq_title_link(tag)
    result = nasdaq_news_dict(dict_list,symbol)
    for i in result:
        insert_nasdaq_many_news(i)

In [206]:
fetch_nasdaq_news('aapl')


-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------
-----------------100%------------------------


In [ ]:
import datetime

now = datetime.datetime.strptime('2023-02-19T18:55:41.000Z', '%Y-%m-%dT%H:%M:%S.000Z')
current_time = now.strftime("%Y-%m-%d %H:%M:%S")
print("Current Time =", current_time)

now = '2023-02-19T18:55:41.000Z'
print(now.replace('T',' ').replace('.000Z',''))


In [3]:
from data.function import News
nasdaq = News('SET')
nasdaq.fetch_news('AOT')